In [ ]:
import openpyxl
import pandas as pd
import datetime
# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
price_df1= pd.read_excel(rf"C:\Users\sudha\Amazon validation script/webscrape_catalog-CA as on {current_date}.xlsx") 
price_df1

In [ ]:
import pandas as pd

# Assuming review_check_df and review_check_df1 are already defined
price_df = pd.concat([price_df1], ignore_index=True)

# # Remove duplicate rows based on the 'ASIN' column
# price_df = price_df.drop_duplicates(subset='ASIN')

# Check the number of rows to ensure correctness
print(f"Number of rows after removing duplicates based on 'ASIN': {price_df.shape[0]}")

price_df

In [ ]:
import pandas as pd

# Assuming your DataFrame is named `df` and the price column is named 'Price'

# Remove newline characters and excessive spaces
price_df['Price'] = price_df['Price'].str.replace(r'\n', '', regex=True)  # Remove newline characters
price_df['Price'] = price_df['Price'].str.replace(r'\s+', '', regex=True)  # Remove any excessive whitespace

# If the price is in a format like "$174..53", we need to handle this
# Replace '..' with a single '.' to correct the format
price_df['Price'] = price_df['Price'].str.replace(r'\.\.', '.', regex=True)
price_df = price_df[['ASIN','Title', 'Price','buybox_owner']]
# Verify the changes
print(price_df[['ASIN','Title', 'Price','buybox_owner']])

In [ ]:
import re
# Define refined regex patterns for 'Sold by'
sold_by_primary_pattern = r"Sold by.*?<span.*?>(?:<a.*?>)?\s*(.*?)\s*<"
sold_by_fallback_pattern = r"Sold by.*?<span.*?>.*?<a.*?>\s*(.*?)\s*</"

ships_from_pattern = r"Ships from.*?<span.*?>(?:<a.*?>)?\s*(.*?)\s*<"


# Function to extract the first match for 'Sold by' with primary and fallback pattern
def extract_sold_by(text, primary_pattern, fallback_pattern):
    if isinstance(text, str):  # Ensure input is a string
        # Check if the primary pattern matches
        primary_match = re.findall(primary_pattern, text, re.DOTALL)
        if primary_match and primary_match[0].strip():  # Ensure match is not empty or just whitespace
            print(f"Primary pattern matched: {primary_match[0]}")  # Log primary match
            return primary_match[0]  # Return the first match from primary pattern
        else:
            # Fallback when primary pattern fails or when value is nan or empty
            print("Primary pattern did not match. Applying fallback pattern.")  # Log fallback use
            fallback_match = re.findall(fallback_pattern, text, re.DOTALL)
            if fallback_match:
                print(f"Fallback pattern matched: {fallback_match[0]}")  # Log fallback match
                return fallback_match[0]  # Return the first match from fallback pattern
    return None

# Function to extract the first match for 'Sold by' and 'Ships from'
def extract_value(text, pattern):
    if isinstance(text, str):  # Ensure input is a string
        matches = re.findall(pattern, text, re.DOTALL)  # Find all matches
        if matches:
            return matches[0]  # Return the first match
    return None


# Apply extraction logic to each row
price_df['Sold by'] = price_df['buybox_owner'].apply(lambda x: extract_sold_by(x, sold_by_primary_pattern, sold_by_fallback_pattern))
price_df['Ships from'] = price_df['buybox_owner'].apply(lambda x: extract_value(x, ships_from_pattern))

price_df= price_df[['ASIN','Title', 'Price','Sold by','Ships from']]
# Display the updated DataFrame

price_df

In [ ]:
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
price_df.to_excel(f'buybox_owner-CA as on {current_date}.xlsx',index=False)